In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Step 1: Create a session and load the page
url1 = 'https://www.nba.com/stats/teams/opponent-shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals'
url2 = 'https://www.nba.com/stats/teams/opponent-shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals'
url3 = 'https://www.nba.com/stats/teams/opponent-shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/opponent-shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals'
url_list = [url1,url2,url3,url4]
def get_tables(url_list):
    data = []
    for url in url_list:
        driver = webdriver.Chrome()
        driver.get(url)

        # Wait for the page to fully load
        driver.implicitly_wait(10)

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())

        driver.close()
        df= dfs[-1]
        drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        df.columns = df.columns.droplevel()
        df = df.drop(columns = drop)
        data.append(df)
    return data

In [2]:
tables = get_tables(url_list)
terms = ['opp_shooting/very_tight.csv','opp_shooting/tight.csv','opp_shooting/open.csv','opp_shooting/wide_open.csv']
jsons =  ['opp_shooting/very_tight.json','opp_shooting/tight.json','opp_shooting/open.json','opp_shooting/wide_open.json']
for i in range(len(terms)):
    df = tables[i]
    df.to_csv(terms[i],index = False)
    df.to_json(jsons[i])


In [3]:
tables[0].to_json(orient="records")

'[{"TEAM":"Utah Jazz","GP":39,"G":39,"Freq%":9.0,"FGM":164,"FGA":314,"FG%":52.2,"eFG%":52.5,"2FG Freq%":8.9,"2FGM":162,"2FGA":310,"2FG%":52.3,"3FG Freq%":0.1,"3PM":2,"3PA":4,"3P%":"50.0"},{"TEAM":"Los Angeles Lakers","GP":37,"G":37,"Freq%":8.3,"FGM":139,"FGA":286,"FG%":48.6,"eFG%":49.1,"2FG Freq%":8.0,"2FGM":136,"2FGA":277,"2FG%":49.1,"3FG Freq%":0.3,"3PM":3,"3PA":9,"3P%":"33.3"},{"TEAM":"Brooklyn Nets","GP":37,"G":37,"Freq%":8.8,"FGM":123,"FGA":284,"FG%":43.3,"eFG%":43.7,"2FG Freq%":8.7,"2FGM":121,"2FGA":281,"2FG%":43.1,"3FG Freq%":0.1,"3PM":2,"3PA":3,"3P%":"66.7"},{"TEAM":"Oklahoma City Thunder","GP":36,"G":36,"Freq%":8.7,"FGM":136,"FGA":283,"FG%":48.1,"eFG%":48.1,"2FG Freq%":8.7,"2FGM":136,"2FGA":283,"2FG%":48.1,"3FG Freq%":0.0,"3PM":0,"3PA":0,"3P%":"-"},{"TEAM":"Atlanta Hawks","GP":37,"G":36,"Freq%":8.2,"FGM":141,"FGA":274,"FG%":51.5,"eFG%":51.5,"2FG Freq%":8.1,"2FGM":141,"2FGA":269,"2FG%":52.4,"3FG Freq%":0.2,"3PM":0,"3PA":5,"3P%":"0.0"},{"TEAM":"Washington Wizards","GP":38,"G":38

In [11]:
len(tables)

4

In [ ]:
tables[1]